In [1]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Model performance evaluation
import evaluate
import numpy as np

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import pipeline
import pandas as pd
# Hugging Face Dataset
from datasets import Dataset
# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline
from huggingface_hub import notebook_login
notebook_login()

#model_path = "mental/mental-bert-base-uncased"

df1 = pd.read_parquet("non_depressed_clean.parquet")
df2 = pd.read_parquet("depression_clean.parquet")

# Append df2 to df1
df_subreddits = pd.concat([df1,df2])



In [4]:
df1['label'] = df1['label'].astype(int)

df2.dtypes

label     int32
value    object
dtype: object

In [5]:
df_subreddits.head(5)

,label,value
0,0,I like how the Red Tie Legion chose a picture ...
1,0,I used to love to mix addys and Valium 10s tho...
2,0,Thanks for the advice man ! Have a great New Y...
3,0,The simple ones feel elegant. The simple ones ...
4,0,"Hopefully ""new games"" doesn't mean ""new ways t..."


In [6]:
import re
import pandas as pd

def preprocessing_df(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text



# Clean the 'value' column using the clean_text function
df_subreddits['value'] = df_subreddits['value'].apply(preprocessing_df)




In [7]:
df_subreddits.info()
df_subreddits['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 3605 entries, 0 to 1705
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   3605 non-null   int32 
 1   value   3605 non-null   object
dtypes: int32(1), object(1)
memory usage: 70.4+ KB


label
0    1899
1    1706
Name: count, dtype: int64

In [8]:
# Read in data
#amz_review = pd.read_csv( 'amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])


In [9]:
from sklearn.model_selection import train_test_split

# Assuming 'appended_df' contains your dataset and 'target_column' is the column containing class labels
X = df_subreddits.drop(columns=['label'])
y = df_subreddits['label']

# Stratified splitting with an 80-20 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create training and testing DataFrames
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Check the number of records in training and testing datasets.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

The training dataset has 2884 records.
The testing dataset has 721 records.


In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
model_class, tokenizer_class, config_class = AutoModelForSequenceClassification,AutoTokenizer,AutoConfig

path= "mental/mental-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(path)



model = AutoModelForSequenceClassification.from_pretrained(path, num_labels = 2)





Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# View tokenizer
tokenizer

RobertaTokenizerFast(name_or_path='mental/mental-roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [13]:
tokenizer.sep_token_id

2

In [14]:
# Mapping between special tokens and their IDs.
print(f'The unknown token is {tokenizer.unk_token} and the ID for the unkown token is {tokenizer.unk_token_id}.')
print(f'The seperator token is {tokenizer.sep_token} and the ID for the seperator token is {tokenizer.sep_token_id}.')
print(f'The pad token is {tokenizer.pad_token} and the ID for the pad token is {tokenizer.pad_token_id}.')
print(f'The sentence level classification token is {tokenizer.cls_token} and the ID for the classification token is {tokenizer.cls_token_id}.')
print(f'The mask token is {tokenizer.mask_token} and the ID for the mask token is {tokenizer.mask_token_id}.')

The unknown token is <unk> and the ID for the unkown token is 3.
The seperator token is </s> and the ID for the seperator token is 2.
The pad token is <pad> and the ID for the pad token is 1.
The sentence level classification token is <s> and the ID for the classification token is 0.
The mask token is <mask> and the ID for the mask token is 50264.


In [15]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["value"], 
                     max_length=32, 
                     truncation=True, 
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/2884 [00:00<?, ? examples/s]

Map:   0%|          | 0/721 [00:00<?, ? examples/s]

In [16]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['value', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2884
})
Dataset({
    features: ['value', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 721
})


In [17]:
len(dataset_train)

2884

In [18]:
dataset_train[0]

{'value': 'hello this year i went deep into depression i think my routine was a lot like yours waking up feeling hopeless thinking about the reasons not to kill myself that very day browsing this subreddit to feel myself validated and to know that im not the only one thinking like this but this subreddit is a downward spiral do yourself a favor if youre depressed and dont come to this subreddit seeing the phrase im gonna kill myself or its done for me doesnt make you feel validated it perpetuates the down spiral\n\nin june my mother asked me youre beyond your control arent you and i answered yes while crying she booked therapy for me because i didnt had the courage then from there life slowly started to be brighter \ni found out that i have a depressive nature and that its ok its how i am and with therapy you can develop strategies to fight it or a better term accept it as a part of you\ni hope that i never will come back to that state of mind but if i return i know ways to fight it\n\

In [19]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./classification_transfer_learning_transformer/",          
    logging_dir='./classification_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=5,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,  
    learning_rate=2e-5,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [20]:
# Number of evaluation modules
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 138 evaluation models in Hugging Face.



['precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'kaggle/ai4code',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'poseval',
 'brier_score',
 'abidlabs/mean_iou',
 'abidlabs/mean_iou2',
 'giulio98/codebleu',
 'mase',
 'mape',
 'smape',
 'dvitel/codebleu',
 'NCSOFT/harim_plus',
 'JP-SystemsX/nDCG',
 'Drunper/metrica_tesi',
 'jpxkqx/peak_signal_to_noise_ratio',
 'jpxkqx/signal_to_reconstruction_error',
 'hpi-dhc/FairEva

In [21]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
   callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

c:\Users\Mirel Agy\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/455 [00:00<?, ?it/s]

{'loss': 0.247, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


  0%|          | 0/23 [00:00<?, ?it/s]

{'eval_loss': 0.08266227692365646, 'eval_accuracy': 0.9764216366158114, 'eval_runtime': 69.0911, 'eval_samples_per_second': 10.436, 'eval_steps_per_second': 0.333, 'epoch': 1.0}
{'loss': 0.0592, 'learning_rate': 1.2e-05, 'epoch': 2.0}


  0%|          | 0/23 [00:00<?, ?it/s]

{'eval_loss': 0.10323011130094528, 'eval_accuracy': 0.9736477115117892, 'eval_runtime': 75.4581, 'eval_samples_per_second': 9.555, 'eval_steps_per_second': 0.305, 'epoch': 2.0}
{'train_runtime': 1945.924, 'train_samples_per_second': 7.41, 'train_steps_per_second': 0.234, 'train_loss': 0.15308823952308068, 'epoch': 2.0}


TrainOutput(global_step=182, training_loss=0.15308823952308068, metrics={'train_runtime': 1945.924, 'train_samples_per_second': 7.41, 'train_steps_per_second': 0.234, 'train_loss': 0.15308823952308068, 'epoch': 2.0})

In [23]:
# Predictions
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
y_test_predict

  0%|          | 0/23 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-2.9904017,  2.8293424],
       [ 3.270552 , -2.8226159],
       [ 3.2402349, -2.8625734],
       ...,
       [-2.9630318,  2.8111506],
       [ 3.2539642, -2.7969892],
       [ 3.266674 , -2.8195107]], dtype=float32), label_ids=array([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
    

In [24]:
# Predicted logits
y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities
y_test_logits[:5]

array([[-2.9904017,  2.8293424],
       [ 3.270552 , -2.8226159],
       [ 3.2402349, -2.8625734],
       [ 3.2840462, -2.8510957],
       [-2.9717913,  2.815731 ]], dtype=float32)

In [25]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_logits)

# First 5 predicted logits
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.00295958, 0.99704045],
       [0.99774677, 0.00225316],
       [0.9977684 , 0.00223159],
       [0.9978393 , 0.00216074],
       [0.0030562 , 0.9969438 ]], dtype=float32)>

In [26]:
# Predicted labels
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted probabilities
y_test_pred_labels[:5]

array([1, 0, 0, 0, 1], dtype=int64)

In [27]:
# Actual labels
y_test_actual_labels = y_test_predict.label_ids

# First 5 predicted probabilities
y_test_actual_labels[:5]

array([1, 0, 0, 0, 1], dtype=int64)

In [28]:
# Trainer evaluate
trainer.evaluate(dataset_train)

  0%|          | 0/91 [00:00<?, ?it/s]

{'eval_loss': 0.05030614137649536,
 'eval_accuracy': 0.9840499306518724,
 'eval_runtime': 284.8165,
 'eval_samples_per_second': 10.126,
 'eval_steps_per_second': 0.32,
 'epoch': 2.0}

In [29]:
# Trainer evaluate
trainer.evaluate(dataset_test)

  0%|          | 0/23 [00:00<?, ?it/s]

{'eval_loss': 0.08266227692365646,
 'eval_accuracy': 0.9764216366158114,
 'eval_runtime': 74.136,
 'eval_samples_per_second': 9.725,
 'eval_steps_per_second': 0.31,
 'epoch': 2.0}

In [30]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

{'f1': 0.9747399702823181}

In [31]:


# Load recall metric
metric_recall = evaluate.load("recall")

# Compute recall metric
metric_recall.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

{'recall': 0.9618768328445748}

In [32]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_transformer/mental_roberta/')

# Save model
trainer.save_model('./sentiment_transfer_learning_transformer/mental_roberta/')

In [33]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_transformer/mental_roberta/")

# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_transformer/mental_roberta/')

In [ ]:
import nlp
import numpy as np
import scipy as sp
import torch
import transformers

import shap

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_path = "./sentiment_transfer_learning_transformer/mental_roberta/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM



# define a prediction function
def f(x):
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length= 512, truncation=True)
            for v in x
        ]
    )
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val




# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)



# explain the model's predictions on IMDB reviews
#imdb_train = nlp.load_dataset("imdb")["train"]
shap_values = explainer(hg_test_data['value'][1:10], fixed_context=1)

shap_values

shap.plots.bar(shap_values.abs.max(0))